In [64]:
# Importamos todas las librerías que vamos a utlizar durante el trabajo

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [65]:
#Cargamos el dataset de 'train', en el cual vamos a trabajar en el modelo.

hosp_train = pd.read_csv('Datasets/hospitalizaciones_train_after_EDA.csv')
hosp_train.drop(columns='Unnamed: 0', inplace=True)
hosp_train

,Available Extra Rooms in Hospital,Department,doctor_name,Age,gender,Type of Admission,Severity of Illness,health_conditions,Stay (in days)
0,4,gynecology,Dr Sophia,41-50,Female,Trauma,Extreme,Diabetes,0
1,4,gynecology,Dr Sophia,31-40,Female,Trauma,Minor,Heart disease,1
2,2,gynecology,Dr Sophia,21-30,Female,Trauma,Extreme,Diabetes,0
3,4,gynecology,Dr Olivia,31-40,Female,Urgent,Moderate,None,0
4,2,anesthesia,Dr Mark,71-80,Male,Trauma,Moderate,Diabetes,1
...,...,...,...,...,...,...,...,...,...
409995,1,gynecology,Dr Sarah,31-40,Female,Trauma,Minor,Asthama,0
409996,5,gynecology,Dr Nathan,21-30,Female,Trauma,Extreme,Other,1
409997,4,gynecology,Dr Sarah,21-30,Female,Emergency,Minor,Asthama,1
409998,10,gynecology,Dr Olivia,21-30,Female,Trauma,Moderate,Other,1


In [66]:
# Para poder probar el modelo, voy a crear tambien un dataframe con la función 'get_dummies', que transforma en binario las columnas categóricas.

hosp_train_dummies = pd.get_dummies(hosp_train)

hosp_train_dummies

,Available Extra Rooms in Hospital,Stay (in days),Department_TB & Chest disease,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,doctor_name_Dr Isaac,doctor_name_Dr John,doctor_name_Dr Mark,...,Type of Admission_Urgent,Severity of Illness_Extreme,Severity of Illness_Minor,Severity of Illness_Moderate,health_conditions_Asthama,health_conditions_Diabetes,health_conditions_Heart disease,health_conditions_High Blood Pressure,health_conditions_None,health_conditions_Other
0,4,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,4,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,2,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,4,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
4,2,1,0,1,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409995,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
409996,5,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
409997,4,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
409998,10,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [67]:
#También creo otro dataframe para poder probar el modelo, esta vez uso OneHotEncoder que codifica las columnas categóricas del dataframe.

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

enc = OneHotEncoder(categories='auto')
categorical_columns = ['Department', 'doctor_name', 'Age',
       'gender', 'Type of Admission', 'Severity of Illness',
       'health_conditions']

ohe_transform = enc.fit_transform(hosp_train[categorical_columns]).toarray()
ohe_columns_name = enc.get_feature_names(categorical_columns)

ohe_hosp_train = pd.DataFrame(ohe_transform, columns=ohe_columns_name)
ohe_hosp_train['Available Extra Rooms in Hospital'] = hosp_train['Available Extra Rooms in Hospital']
ohe_hosp_train.columns

c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Index(['Department_TB & Chest disease', 'Department_anesthesia',
       'Department_gynecology', 'Department_radiotherapy',
       'Department_surgery', 'doctor_name_Dr Isaac', 'doctor_name_Dr John',
       'doctor_name_Dr Mark', 'doctor_name_Dr Nathan', 'doctor_name_Dr Olivia',
       'doctor_name_Dr Sam', 'doctor_name_Dr Sarah', 'doctor_name_Dr Simon',
       'doctor_name_Dr Sophia', 'Age_0-10', 'Age_11-20', 'Age_21-30',
       'Age_31-40', 'Age_41-50', 'Age_51-60', 'Age_61-70', 'Age_71-80',
       'Age_81-90', 'Age_91-100', 'gender_Female', 'gender_Male',
       'gender_Other', 'Type of Admission_Emergency',
       'Type of Admission_Trauma', 'Type of Admission_Urgent',
       'Severity of Illness_Extreme', 'Severity of Illness_Minor',
       'Severity of Illness_Moderate', 'health_conditions_Asthama',
       'health_conditions_Diabetes', 'health_conditions_Heart disease',
       'health_conditions_High Blood Pressure', 'health_conditions_None',
       'health_conditions_Other', 'Av

In [68]:
#También, a modo de prueba para el modelo, voy a utilizar LabelEncoder

from sklearn.preprocessing import LabelEncoder

label_encoder = hosp_train.copy()

label_encoder[['Available Extra Rooms in Hospital', 'Department', 'doctor_name', 'Age',
       'gender', 'Type of Admission', 'Severity of Illness',
       'health_conditions']] = label_encoder[['Available Extra Rooms in Hospital', 'Department', 'doctor_name', 'Age',
       'gender', 'Type of Admission', 'Severity of Illness',
       'health_conditions']].apply(LabelEncoder().fit_transform)
    

label_encoder

,Available Extra Rooms in Hospital,Department,doctor_name,Age,gender,Type of Admission,Severity of Illness,health_conditions,Stay (in days)
0,4,2,8,4,0,1,0,1,0
1,4,2,8,3,0,1,1,2,1
2,2,2,8,2,0,1,0,1,0
3,4,2,4,3,0,2,2,4,0
4,2,1,2,7,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...
409995,1,2,6,3,0,1,1,0,0
409996,5,2,3,2,0,1,0,5,1
409997,4,2,6,2,0,0,1,0,1
409998,10,2,4,2,0,1,2,5,1


In [69]:
#Esta función realiza un entrenamiento de un modelo DecisionTreeClassifier con los datos propuestos.

def test_decision_tree_metrics(x,y,md:int,ts:float,criterio:str):
    decision_tree = DecisionTreeClassifier(max_depth = md, random_state = 42, criterion=criterio)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=ts, random_state=42)
    decision_tree.fit(x_train.values,y_train.values)
    y_pred = decision_tree.predict(x_test)
    result = (md,accuracy_score(y_test,y_pred),recall_score(y_test,y_pred))
    
    return result

In [70]:
#En este bloque de código se declaran las variables que se van a utilizar tanto para entrenar como para testear el modelo de árbol de decision.

x_ohe = ohe_hosp_train[['Department_TB & Chest disease', 'Department_anesthesia',
       'Department_gynecology', 'Department_radiotherapy',
       'Department_surgery', 'doctor_name_Dr Isaac', 'doctor_name_Dr John',
       'doctor_name_Dr Mark', 'doctor_name_Dr Nathan', 'doctor_name_Dr Olivia',
       'doctor_name_Dr Sam', 'doctor_name_Dr Sarah', 'doctor_name_Dr Simon',
       'doctor_name_Dr Sophia', 'Age_0-10', 'Age_11-20', 'Age_21-30',
       'Age_31-40', 'Age_41-50', 'Age_51-60', 'Age_61-70', 'Age_71-80',
       'Age_81-90', 'Age_91-100', 'gender_Female', 'gender_Male',
       'gender_Other', 'Type of Admission_Emergency',
       'Type of Admission_Trauma', 'Type of Admission_Urgent',
       'Severity of Illness_Extreme', 'Severity of Illness_Minor',
       'Severity of Illness_Moderate', 'health_conditions_Asthama',
       'health_conditions_Diabetes', 'health_conditions_Heart disease',
       'health_conditions_High Blood Pressure', 'health_conditions_None',
       'health_conditions_Other', 'Available Extra Rooms in Hospital']]

x_dummies = hosp_train_dummies[['Available Extra Rooms in Hospital',
       'Department_TB & Chest disease', 'Department_anesthesia',
       'Department_gynecology', 'Department_radiotherapy',
       'Department_surgery', 'doctor_name_Dr Isaac', 'doctor_name_Dr John',
       'doctor_name_Dr Mark', 'doctor_name_Dr Nathan', 'doctor_name_Dr Olivia',
       'doctor_name_Dr Sam', 'doctor_name_Dr Sarah', 'doctor_name_Dr Simon',
       'doctor_name_Dr Sophia', 'Age_0-10', 'Age_11-20', 'Age_21-30',
       'Age_31-40', 'Age_41-50', 'Age_51-60', 'Age_61-70', 'Age_71-80',
       'Age_81-90', 'Age_91-100', 'gender_Female', 'gender_Male',
       'gender_Other', 'Type of Admission_Emergency',
       'Type of Admission_Trauma', 'Type of Admission_Urgent',
       'Severity of Illness_Extreme', 'Severity of Illness_Minor',
       'Severity of Illness_Moderate', 'health_conditions_Asthama',
       'health_conditions_Diabetes', 'health_conditions_Heart disease',
       'health_conditions_High Blood Pressure', 'health_conditions_None',
       'health_conditions_Other']]

x_le = label_encoder[['Available Extra Rooms in Hospital', 'Department', 'doctor_name', 'Age',
       'gender', 'Type of Admission', 'Severity of Illness',
       'health_conditions']]

y = hosp_train[['Stay (in days)']]

In [71]:
#En este bloque de código voy a entrenar el modelo de árbol de decisión con la variable 'x_dummies' que serían las columnas categóricas de el dataframe hosp_train pasadas
#por el método de pandas 'get_dummies'. La variable 'y' no cambia.

metrics_dummies_entropy = []

metrics_dummies = []

metrics_dummies_losg_loss = []

for i in range(1,30):
    metrics_dummies_entropy.append(test_decision_tree_metrics(x_dummies,y,i,0.30,'entropy'))

for i in range(1,30):
    metrics_dummies.append(test_decision_tree_metrics(x_dummies,y,i,0.30,'gini'))

for i in range(1,30):
    metrics_dummies_losg_loss.append(test_decision_tree_metrics(x_dummies,y,i,0.30,'log_loss'))

c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without featur

In [72]:
#En este bloque de código voy a entrenar el modelo de árbol de decisión con la variable 'x_le' la misma contendría las columnas categóricas de el dataframe hosp_train pasadas
#por el método LabelEncoder, con fines de testeo del modelo. La variable 'y' no cambia.

metrics_le = []

metrics_le_entropy = []

metrics_le_log_loss = []

for i in range(1,30):
    metrics_le.append(test_decision_tree_metrics(x_le,y,i,0.30,'gini'))

for i in range(1,30):
    metrics_le_entropy.append(test_decision_tree_metrics(x_le,y,i,0.30,'entropy'))

for i in range(1,30):
    metrics_le_log_loss.append(test_decision_tree_metrics(x_le,y,i,0.30,'log_loss'))

c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without featur

In [73]:
#En este bloque de código voy a entrenar el modelo de árbol de decisión con la variable 'x_ohe' la misma contendría las columnas categóricas de el dataframe hosp_train pasadas
#por el método OneHotEncoder, con fines de testeo del modelo. La variable 'y' no cambia.

metrics_ohe_entropy = []

metrics_ohe_log_loss = []

metrics_ohe = []

for i in range(1,30):
    metrics_ohe_entropy.append(test_decision_tree_metrics(x_ohe,y,i,0.30,'entropy'))

for i in range(1,30):
    metrics_ohe.append(test_decision_tree_metrics(x_ohe,y,i,0.30,'gini'))

for i in range(1,30):
    metrics_ohe_log_loss.append(test_decision_tree_metrics(x_ohe,y,i,0.30,'log_loss'))

c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without featur

In [74]:
#Con esta función transformo los resultados de los modelos en un dataframe, para un mejor análisis de los datos.

def create_df_from_tuple_list(list,df_name):
    depth = []
    acc = []
    recall = []
    
    for max_depth in list:
        depth.append(max_depth[0])
    
    for accu in list:
        acc.append(accu[1])
    
    for rcc in list:
        recall.append(rcc[2])
    
    result = pd.DataFrame({'max_depth': depth, 'accuracy': acc, 'recall': recall, 'name_list': df_name})
    
    return result.sort_values(by='accuracy', ascending=False)

In [75]:
#En este bloque de código utilizo la función que devuelve los resultados de los testeos de los modelos y genera un dataframe para posteriormente unir todos los resultados
#en un solo dataframe y después filtrarlo por el mejor resultado en 'accuracy', y el mejor resultado se encuentra cuando pasamos al dataframe la función 'get_dummies' y el
#parámetro 'entrophy' en el árbol de decisión. 

lists_of_dfs = [create_df_from_tuple_list(metrics_dummies_entropy, 'metrics_dummies_entropy'), create_df_from_tuple_list(metrics_dummies, 'metrics_dummies'), create_df_from_tuple_list(metrics_dummies_losg_loss, 'metrics_dummies_losg_loss'), create_df_from_tuple_list(metrics_le, 'metrics_le'), create_df_from_tuple_list(metrics_le_entropy, 'metrics_le_entropy'), create_df_from_tuple_list(metrics_le_log_loss, 'metrics_le_log_loss'), create_df_from_tuple_list(metrics_ohe, 'metrics_ohe'), create_df_from_tuple_list(metrics_ohe_entropy, 'metrics_ohe_entropy'), create_df_from_tuple_list(metrics_ohe_log_loss, 'metrics_ohe_entropy')]

dfs_result = pd.concat(lists_of_dfs)
dfs_result.head(5)

,max_depth,accuracy,recall,name_list
17,18,0.697902,0.848427,metrics_dummies_entropy
16,17,0.697683,0.844560,metrics_dummies_entropy
19,20,0.697350,0.854593,metrics_dummies_entropy
15,16,0.697057,0.857977,metrics_dummies_entropy
21,22,0.696992,0.855482,metrics_dummies_entropy


Creo que el hecho de borrar algunas columnas que a mi criterio no debían estar afectó al accuracy, mientras que el recall me parece que se encuentra en un punto muy bueno. Creo que es mejor contar con menos accuracy pero tener una mejor calidad de datos.

In [76]:
#Cargo el dataset que se va a usar posteriormente para el testeo y borro las columnas que eliminé en su contraparte de entrenamiento.

hosp_test = pd.read_csv('Datasets/hospitalizaciones_test.csv') 
hosp_test.drop(columns=['Ward_Facility_Code', 'staff_available', 'Visitors with Patient', 'Insurance', 'Admission_Deposit','patientid'], inplace=True)
hosp_test

,Available Extra Rooms in Hospital,Department,doctor_name,Age,gender,Type of Admission,Severity of Illness,health_conditions
0,3,gynecology,Dr Olivia,21-30,Female,Trauma,Moderate,Other
1,2,TB & Chest disease,Dr John,51-60,Female,Trauma,Moderate,Asthama
2,2,radiotherapy,Dr John,51-60,Other,Trauma,Moderate,None
3,2,gynecology,Dr Olivia,21-30,Female,Trauma,Moderate,None
4,2,anesthesia,Dr John,51-60,Female,Trauma,Extreme,Diabetes
...,...,...,...,...,...,...,...,...
89995,4,gynecology,Dr Sarah,11-20,Female,Trauma,Minor,High Blood Pressure
89996,13,gynecology,Dr Olivia,31-40,Female,Emergency,Moderate,Other
89997,2,gynecology,Dr Sarah,11-20,Female,Emergency,Moderate,High Blood Pressure
89998,2,radiotherapy,Dr John,61-70,Female,Trauma,Extreme,Diabetes


In [77]:
#Utilizo la función get_dummies para poder transformar las columnas categóricas a 1 o 0

hosp_test_dummies = pd.get_dummies(hosp_test)
hosp_test_dummies

,Available Extra Rooms in Hospital,Department_TB & Chest disease,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,doctor_name_Dr Isaac,doctor_name_Dr John,doctor_name_Dr Mark,doctor_name_Dr Nathan,...,Type of Admission_Urgent,Severity of Illness_Extreme,Severity of Illness_Minor,Severity of Illness_Moderate,health_conditions_Asthama,health_conditions_Diabetes,health_conditions_Heart disease,health_conditions_High Blood Pressure,health_conditions_None,health_conditions_Other
0,3,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,2,1,0,0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
2,2,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
3,2,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,2,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,4,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
89996,13,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
89997,2,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
89998,2,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0


In [78]:
#Creo las variables de entrenamiento y testeo.


x_hosp_test = hosp_train_dummies[['Available Extra Rooms in Hospital',
       'Department_TB & Chest disease', 'Department_anesthesia',
       'Department_gynecology', 'Department_radiotherapy',
       'Department_surgery', 'doctor_name_Dr Isaac', 'doctor_name_Dr John',
       'doctor_name_Dr Mark', 'doctor_name_Dr Nathan', 'doctor_name_Dr Olivia',
       'doctor_name_Dr Sam', 'doctor_name_Dr Sarah', 'doctor_name_Dr Simon',
       'doctor_name_Dr Sophia', 'Age_0-10', 'Age_11-20', 'Age_21-30',
       'Age_31-40', 'Age_41-50', 'Age_51-60', 'Age_61-70', 'Age_71-80',
       'Age_81-90', 'Age_91-100', 'gender_Female', 'gender_Male',
       'gender_Other', 'Type of Admission_Emergency',
       'Type of Admission_Trauma', 'Type of Admission_Urgent',
       'Severity of Illness_Extreme', 'Severity of Illness_Minor',
       'Severity of Illness_Moderate', 'health_conditions_Asthama',
       'health_conditions_Diabetes', 'health_conditions_Heart disease',
       'health_conditions_High Blood Pressure', 'health_conditions_None',
       'health_conditions_Other']]

y_hosp_test = hosp_train_dummies[['Stay (in days)']]

x_test = hosp_test_dummies[['Available Extra Rooms in Hospital', 'Department_TB & Chest disease',
       'Department_anesthesia', 'Department_gynecology',
       'Department_radiotherapy', 'Department_surgery', 'doctor_name_Dr Isaac',
       'doctor_name_Dr John', 'doctor_name_Dr Mark', 'doctor_name_Dr Nathan',
       'doctor_name_Dr Olivia', 'doctor_name_Dr Sam', 'doctor_name_Dr Sarah',
       'doctor_name_Dr Simon', 'doctor_name_Dr Sophia', 'Age_0-10',
       'Age_11-20', 'Age_21-30', 'Age_31-40', 'Age_41-50', 'Age_51-60',
       'Age_61-70', 'Age_71-80', 'Age_81-90', 'Age_91-100', 'gender_Female',
       'gender_Male', 'gender_Other', 'Type of Admission_Emergency',
       'Type of Admission_Trauma', 'Type of Admission_Urgent',
       'Severity of Illness_Extreme', 'Severity of Illness_Minor',
       'Severity of Illness_Moderate', 'health_conditions_Asthama',
       'health_conditions_Diabetes', 'health_conditions_Heart disease',
       'health_conditions_High Blood Pressure', 'health_conditions_None',
       'health_conditions_Other']]

In [79]:
#Entrenamiento del modelo y predicción.

arbol_decision = DecisionTreeClassifier(max_depth = 18, random_state = 42, criterion='entropy')

arbol_decision.fit(x_hosp_test,y_hosp_test)

y_pred = arbol_decision.predict(x_test)
y_pred = pd.DataFrame({'pred':y_pred})
y_pred


,pred
0,1
1,1
2,1
3,1
4,1
...,...
89995,1
89996,1
89997,1
89998,1


In [91]:
y_pred.to_csv('fcori47.csv', index=False)

In [81]:
#Ahora voy a probar con otro dataset que cree, en el que hay más Falsos Positivos en los datos.
hosp_train_fp = pd.read_csv('Datasets/hospitalizaciones_train_fp.csv')
hosp_train_fp.drop(columns='Unnamed: 0', inplace=True)
hosp_train_fp

,Available Extra Rooms in Hospital,Department,doctor_name,Age,gender,Type of Admission,Severity of Illness,health_conditions,Stay (in days)
0,4,gynecology,Dr Sophia,41-50,Female,Trauma,Extreme,Diabetes,0
1,4,gynecology,Dr Sophia,31-40,Female,Trauma,Minor,Heart disease,1
2,2,gynecology,Dr Sophia,21-30,Female,Trauma,Extreme,Diabetes,0
3,4,gynecology,Dr Olivia,31-40,Female,Urgent,Moderate,None,0
4,2,anesthesia,Dr Mark,71-80,Male,Trauma,Moderate,Diabetes,1
...,...,...,...,...,...,...,...,...,...
409995,1,gynecology,Dr Sarah,31-40,Female,Trauma,Minor,Asthama,0
409996,5,gynecology,Dr Nathan,21-30,Female,Trauma,Extreme,Other,1
409997,4,gynecology,Dr Sarah,21-30,Female,Emergency,Minor,Asthama,1
409998,10,gynecology,Dr Olivia,21-30,Female,Trauma,Moderate,Other,1


In [82]:
#Utilizo la función de pandas 'get_dummies' para transformar las columnas categóricas a 0 o 1.

hosp_train_fp_dummies = pd.get_dummies(hosp_train_fp)
hosp_train_fp_dummies 

,Available Extra Rooms in Hospital,Stay (in days),Department_TB & Chest disease,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,doctor_name_Dr Isaac,doctor_name_Dr John,doctor_name_Dr Mark,...,Type of Admission_Urgent,Severity of Illness_Extreme,Severity of Illness_Minor,Severity of Illness_Moderate,health_conditions_Asthama,health_conditions_Diabetes,health_conditions_Heart disease,health_conditions_High Blood Pressure,health_conditions_None,health_conditions_Other
0,4,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,4,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,2,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,4,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
4,2,1,0,1,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409995,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
409996,5,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
409997,4,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
409998,10,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [83]:
#Entrenamiento del modelo nuevamente con el dataframe que tiene más falsos positivos.

x_fp = hosp_train_fp_dummies[['Available Extra Rooms in Hospital',
       'Department_TB & Chest disease', 'Department_anesthesia',
       'Department_gynecology', 'Department_radiotherapy',
       'Department_surgery', 'doctor_name_Dr Isaac', 'doctor_name_Dr John',
       'doctor_name_Dr Mark', 'doctor_name_Dr Nathan', 'doctor_name_Dr Olivia',
       'doctor_name_Dr Sam', 'doctor_name_Dr Sarah', 'doctor_name_Dr Simon',
       'doctor_name_Dr Sophia', 'Age_0-10', 'Age_11-20', 'Age_21-30',
       'Age_31-40', 'Age_41-50', 'Age_51-60', 'Age_61-70', 'Age_71-80',
       'Age_81-90', 'Age_91-100', 'gender_Female', 'gender_Male',
       'gender_Other', 'Type of Admission_Emergency',
       'Type of Admission_Trauma', 'Type of Admission_Urgent',
       'Severity of Illness_Extreme', 'Severity of Illness_Minor',
       'Severity of Illness_Moderate', 'health_conditions_Asthama',
       'health_conditions_Diabetes', 'health_conditions_Heart disease',
       'health_conditions_High Blood Pressure', 'health_conditions_None',
       'health_conditions_Other']]

y_fp = hosp_train_fp_dummies[['Stay (in days)']]

In [84]:
#Probamos el modelo nuevamente con el dataframe enfocado en los Falsos Positivos, con distintos parámetros.

metrics_dummies_fp_entropy = []

metrics_fp_dummies = []

metrics_dummies_fp_losg_loss = []

for i in range(1,30):
    metrics_dummies_fp_entropy.append(test_decision_tree_metrics(x_fp,y_fp,i,0.30,'entropy'))

for i in range(1,30):
    metrics_fp_dummies.append(test_decision_tree_metrics(x_fp,y_fp,i,0.30,'gini'))

for i in range(1,30):
    metrics_dummies_fp_losg_loss.append(test_decision_tree_metrics(x_fp,y_fp,i,0.30,'log_loss'))

c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\elbue\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without featur

In [85]:
#M'etricas del modelo anterior

dfs_result.head(5)

,max_depth,accuracy,recall,name_list
17,18,0.697902,0.848427,metrics_dummies_entropy
16,17,0.697683,0.844560,metrics_dummies_entropy
19,20,0.697350,0.854593,metrics_dummies_entropy
15,16,0.697057,0.857977,metrics_dummies_entropy
21,22,0.696992,0.855482,metrics_dummies_entropy


In [90]:
#Métricas nuevo modelo

create_df_from_tuple_list(metrics_dummies_fp_entropy,'metrics_fp_dummies_entropy').head(5)

,max_depth,accuracy,recall,name_list
16,17,0.699805,0.849524,metrics_fp_dummies_entropy
15,16,0.699602,0.845706,metrics_fp_dummies_entropy
18,19,0.699317,0.855832,metrics_fp_dummies_entropy
20,21,0.698984,0.856705,metrics_fp_dummies_entropy
26,27,0.698902,0.854946,metrics_fp_dummies_entropy


Como vemos, si bien no es por mucha diferencia, el dataframe enfocado a los Falsos Positivos mejora el modelo, por lo que es el modelo que voy a utilizar.

In [95]:
arbol_decision_2 = DecisionTreeClassifier(max_depth = 17, random_state = 42, criterion='entropy')

arbol_decision_2.fit(x_fp,y_fp)

y_pred_2 = arbol_decision.predict(x_test)
y_pred_2 = pd.DataFrame({'pred':y_pred_2})

In [97]:
y_pred_2.to_csv('fcori47.csv', index=False)